# Mitochondria segmentation in EM with 2D U-Net

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import os
import sys
from skimage.io import imread                                                   
from PIL import Image
import numpy as np
from tqdm import tqdm


In [ ]:
!wget https://github.com/danifranco/EM_Image_Segmentation/archive/refs/tags/v1.0.tar.gz
!tar -xzvf v1.0.tar.gz

Install dependencies:

In [ ]:
!pip install git+https://github.com/aleju/imgaug.git

Download the EPFL Hippocampus dataset (Lucchi dataset)

In [ ]:
if not os.path.exists('training.tif'):
    !wget https://documents.epfl.ch/groups/c/cv/cvlab-unit/www/data/%20ElectronMicroscopy_Hippocampus/training.tif
if not os.path.exists('training_groundtruth.tif'):
    !wget https://documents.epfl.ch/groups/c/cv/cvlab-unit/www/data/%20ElectronMicroscopy_Hippocampus/training_groundtruth.tif
if not os.path.exists('testing.tif'):
    !wget https://documents.epfl.ch/groups/c/cv/cvlab-unit/www/data/%20ElectronMicroscopy_Hippocampus/testing.tif
if not os.path.exists('testing_groundtruth.tif'):
    !wget https://documents.epfl.ch/groups/c/cv/cvlab-unit/www/data/%20ElectronMicroscopy_Hippocampus/testing_groundtruth.tif

## Prepare dataset in correct folders readable by the template 

In [ ]:
!mkdir -p dataset/train/x dataset/train/y dataset/test/x dataset/test/y
train_save_data = 'dataset/train/x'
train_save_mask = 'dataset/train/y'
test_save_data = 'dataset/test/x'
test_save_mask = 'dataset/test/y'

**Train data**:

In [ ]:
X_train = imread('training.tif')
Y_train = imread('training_groundtruth.tif')

In [ ]:
d = len(str(X_train.shape[0]))
for i in tqdm(range(X_train.shape[0])):
    # Data
    im = Image.fromarray((X_train[i]).astype(np.uint8))
    im = im.convert('L')
    f = os.path.join(train_save_data , "x_" + str(i).zfill(d) + '.tif')
    im.save(f)

    # Mask
    im = Image.fromarray((Y_train[i]).astype(np.uint8))
    im = im.convert('L')
    f = os.path.join(train_save_mask , "y_" + str(i).zfill(d) + '.tif')
    im.save(f)

In [ ]:
del X_train, Y_train

**Test data**:

In [ ]:
X_test = imread('testing.tif')
Y_test = imread('testing_groundtruth.tif')

In [ ]:
d = len(str(X_test.shape[0]))
for i in tqdm(range(X_test.shape[0])):
    # Data
    im = Image.fromarray((X_test[i]).astype(np.uint8))
    im = im.convert('L')
    f = os.path.join(test_save_data , "x_" + str(i).zfill(d) + '.tif')
    im.save(f)

    # Mask
    im = Image.fromarray((Y_test[i]).astype(np.uint8))
    im = im.convert('L')
    f = os.path.join(test_save_mask , "y_" + str(i).zfill(d) + '.tif')
    im.save(f)

In [ ]:
del X_test, Y_test

Check data samples:

In [ ]:
train_x_sample = imread(os.path.join(train_save_data, 'x_000.tif')) 
train_y_sample = imread(os.path.join(train_save_mask, 'y_000.tif')) 
test_x_sample = imread(os.path.join(test_save_data, 'x_000.tif')) 
test_y_sample = imread(os.path.join(test_save_mask, 'y_000.tif')) 

plt.figure(figsize=(20,20))
plt.subplot(1, 4, 1)    
plt.imshow(train_x_sample, cmap='gray')
plt.subplot(1, 4, 2)    
plt.imshow(train_y_sample, cmap='gray')
plt.subplot(1, 4, 3)    
plt.imshow(test_x_sample, cmap='gray')
plt.subplot(1, 4, 4)    
plt.imshow(test_y_sample, cmap='gray')
plt.show()

del train_x_sample, train_y_sample, test_x_sample, test_y_sample

# Run the template!

In [ ]:
# 1st arg: absolute path to the repo code  -> '/content/EM_Image_Segmentation-1.0'
# 2nd arg: absolute path to the dataset -> '/content/dataset'
# 3rd arg: absolute path where the output data will be generated -> '/content/out_dir'
# 4th arg (--id) : a string to identify the job -> 'test_job' 
# 5th arg (--rid) : number that should be increased when one need to run the same job multiple times -> 0
# 6th arg (--gpu): number of the GPU to run the job in (according to 'nvidia-smi' command) -> 0

# Select one of them. V0 corresponds to the hyperarameters setting presented by Cheng et. al and V1 is our best setting. 
#template='cheng_3D_template_V0.py'
template='cheng_3D_template_V1.py'

!python -u 'EM_Image_Segmentation-1.0/sota_implementations/cheng_2017/'$template '/content/EM_Image_Segmentation-1.0' '/content/dataset' '/content/out_dir' --id 'test_job' --rid 0 --gpu 0

# Check results

In [ ]:
# Visualize training curves 
plt.figure(figsize=(50,50))
plt.subplot(1, 4, 1)    
plt.imshow(imread("/content/out_dir/results/test_job_0/charts/test_job_0_loss.png")) 
plt.subplot(1, 4, 2)    
plt.imshow(imread("/content/out_dir/results/test_job_0/charts/test_job_0_jaccard_index.png"))

In [ ]:
# Load some predicted images 
pred = []
pred.append(imread('/content/out_dir/results/test_job_0/full_no_binarized/y_000.png'))
pred.append(imread('/content/out_dir/results/test_job_0/full_8ensemble/y_000.png'))
pred.append(imread('/content/out_dir/results/test_job_0/full_zfil/y_000.png'))

pred.append(imread('/content/out_dir/results/test_job_0/full_no_binarized/y_055.png'))
pred.append(imread('/content/out_dir/results/test_job_0/full_8ensemble/y_055.png'))
pred.append(imread('/content/out_dir/results/test_job_0/full_zfil/y_055.png'))

gt = []
gt.append(imread(os.path.join(test_save_mask, 'y_000.tif'))) 
gt.append(imread(os.path.join(test_save_mask, 'y_055.tif')))

lbl = []
lbl.append("Predicted full-size image")
lbl.append("Ensamble")
lbl.append("Ensamble + Z-Filtering")
lbl.append("Ground Truth")

In [ ]:
# Show predicted images
for i in range(2):
    plt.figure(figsize=(30,30))
    for j in range(4):
        plt.subplot(i+1, 4, j+1)
        plt.title(lbl[j])
        if j != 3:
            plt.imshow(pred[j+(3*i)], cmap='gray')            
        else:
            plt.imshow(gt[i], cmap='gray')
    plt.show()